In [ ]:
import sys
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from toolkit.common.constants import *
from toolkit.cars.car_configuration import Car
from toolkit.mmd import MMD
from toolkit.steady_state_solver.iterative import Iterative_Solver
from toolkit.common.maths import interpolate, sa_lut

In [ ]:
size = 30
v_avg = 17
long_g = 0
samples = np.linspace(0.95, 1.55, 7)

In [ ]:


max_beta = 30
max_delta = 30
use_lin = True

# tool for graphing balance as a function of velocity. mmds must be sorted in ascending order by velocity
def plot_accel_vs_vel(mmd_list, save_html = False, mmd_list2 = None):
    accs = []
    bals = []
    saccs = []
    sbals = []
    vels = []
    
    #mmds.sort(key=lambda x:x.v_avg)
    # you must do this before calling the function
    
    for m in mmd_list:
        res = m.calculate_max_acc()
        steady = res[0]
        unsteady = res[1]
        #steady, unsteady = m.calculate_max_acc()
        accs.append(unsteady[0])
        bals.append(unsteady[1])
        saccs.append(steady[0])
        sbals.append(steady[1])
        vels.append(m.v_avg)
        
    fig = go.Figure()
    fig.update_xaxes(title_text="Velocity (m/s)", range=[min(vels), max(vels)])
    fig.update_yaxes(title_text="Lateral Acceleration (G)", range=[-1, max(accs)+0.5])
    fig.add_trace(
        go.Scatter(
            x = vels,
            y = saccs,
            mode ='lines',
            marker = dict(color='blue'),
            name = 'Steady-State'
        ))
    fig.add_trace(
        go.Scatter(
            x = vels,
            y = accs,
            mode ='lines',
            marker = dict(color='LightSkyBlue'),
            name = 'Absolute'
        )
    )
    fig.add_trace(
        go.Scatter(
            x = vels,
            y = bals,
            mode = 'lines',
            marker = dict(color='purple'),
            name = 'Yaw Coefficient'
        )
    )
    
    if mmd_list2 is not None:
        print('Comparing to alt config...')
        accs2 = []
        bals2 = []
        saccs2 = []
        sbals2 = []
        vels2 = []
        for m in mmd_list2:
            res = m.calculate_max_acc()
            steady = res[0]
            unsteady = res[1]
            #steady, unsteady = m.calculate_max_acc()
            accs2.append(unsteady[0])
            bals2.append(unsteady[1])
            saccs2.append(steady[0])
            sbals2.append(steady[1])
            vels2.append(m.v_avg)
        fig.add_trace(
            go.Scatter(
                x = vels2,
                y = saccs2,
                mode ='lines',
                marker = dict(color='red'),
                name = 'Steady-State 2'
            ))
        fig.add_trace(
            go.Scatter(
                x = vels2,
                y = accs2,
                mode ='lines',
                marker = dict(color='pink'),
                name = 'Absolute 2'
            ))
        fig.add_trace(
            go.Scatter(
                x = vels2,
                y = bals2,
                mode ='lines',
                marker = dict(color='DarkSlateGrey'),
                name = 'Balance 2'
            ))
        
    fig.update_layout(title_text = "Peak Grip and Balance vs Velocity")
    fig.show()
    
    if save_html:
        # Ensure the directory exists
        output_dir = r".\\MMDs"
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        import time
        t = time.time()
        fig.write_html(f"{output_dir}\PeakGripVsVel{t:.0f}.html")
        


In [ ]:
# default car config
f_ack = "nonlinear"
c_type = "combined"
a_type = "complex"
t_type = "complexfast"

def gen_car(front_ackermann = f_ack, camber_type = c_type, aero_type = a_type, toe_type = t_type, front_track = 46.6*IN_TO_M, rear_track = 46*IN_TO_M, name = "", kfb=None, krb=None, kfr=None, krr=None):
    car = Car(front_ackermann = front_ackermann, camber_type = camber_type, aero_type = aero_type, toe_type = toe_type, front_track=front_track, rear_track=rear_track)
    car.temp_geo = True
    if kfb is not None:
        car.k_f_b = kfb
        # car.k_f_b = kfb * LB_PER_IN_TO_N_PER_M
    if krb is not None:
        car.k_r_b = krb
        # car.k_r_b = krb * LB_PER_IN_TO_N_PER_M
    if kfr is not None:
        car.k_f = kfr
        # car.k_f = kfr * FTLB_TO_NM
    if krr is not None:
        car.k_r = krr
        # car.k_r = krr * FTLB_TO_NM
    car.update_car() # idk if needed twice
    car.set_lltd()
    car.update_car()
    solver = Iterative_Solver(tangent_effects = True)
    mmd = MMD(car, solver=solver, name = name)
    return car, solver, mmd

In [ ]:
# 2D Bump Rate Sweep
# samples = np.linspace(104, 170, 7)
# mmds = np.full((len(samples), len(samples)), None)
# accs = mmds
# bals = mmds
# stabs_s = mmds
# stabs_d = mmds
# ctrls_s = mmds
# ctrls_d = mmds

# for i, fb in enumerate(samples):
#     for j, rb in enumerate(samples):
#         print(f"Progress: {i+j}/{i*j}")
#         car, _, mmd = gen_car(name = f"FB: {fb}, RB: {rb}")
#         car.k_f_b = fb * LB_PER_IN_TO_N_PER_M
#         car.k_r_b = rb * LB_PER_IN_TO_N_PER_M
#         mmd.mmd_sweep(v_avg, lin_space=use_lin, max_beta=max_beta, max_delta=max_delta, size=size, mu=0.65, long_g=long_g)
#         mmd.clear_high_sa(sa_lut(mmd.v_avg))
#         mmds[i][j] = mmd
#         stabs_s[i][j], stabs_d[i][j], _, _ = mmd.calculate_important_control_values(mode="Stability")
#         ctrls_s[i][j], ctrls_d[i][j], _, _ = mmd.calculate_important_control_values(mode="Control")
#         a = mmd.calculate_max_acc()[1]
#         accs[i][j] = a[0]
#         bals[i][j] = a[1]
        

In [ ]:
from joblib import Parallel, delayed
import numpy as np
from tqdm import tqdm

# sweep domains
WIDTH = 7
# k_bumps = np.linspace(104*2, 140*2, 3)
k_bumps = np.array([110*2])
# k_bumps = np.linspace(104*2, 180*2, WIDTH)
k_bumps = k_bumps * LB_PER_IN_TO_N_PER_M
k_rolls = np.linspace(220*.73, 370*.73, WIDTH)
# k_rolls = np.linspace(164, 285, WIDTH)
k_rolls = k_rolls * FTLB_TO_NM
long_gs = np.array([-3.5, 0, 3.5]) # do not make this array more than 3 elements

# KNOW: THIS TAKES A REALLY LONG TIME
# Takes approximately (WIDTH^4)*2 seconds.
# i.e., for WIDTH = 3, 3min. for WIDTH = 5, 21min. for WIDTH = 7, 80min.


# Define the worker function
def process_combination(i, j, k, l, n, kfb, krb, kfr, krr, v, long):
    # print("Starting...")
    car, _, mmd = gen_car(name=f"KFB: {kfb}, KRB: {krb}, KFR: {kfr}, KRR: {krr}", kfb=kfb, krb=krb, kfr=kfr, krr=krr)
    # Perform operations on mmd
    mmd.mmd_sweep(v, lin_space=True, max_beta=30, max_delta=30, size=30, mu=0.65, long_g=long)
    mmd.clear_high_sa(sa_lut(mmd.v_avg))
    
    # TEST
    # Calculate stability and control values
    stab_s, stab_d, _, _ = mmd.calculate_important_control_values(mode="Stability", resamples=201)
    ctrl_s, ctrl_d, _, _ = mmd.calculate_important_control_values(mode="Control", resamples=301, force_zero_cn = True)
    a = mmd.calculate_max_acc()[1]
    
    # Return the results (without control values calculation)
    return i, j, k, l, n, mmd, stab_s, stab_d, ctrl_s, ctrl_d, a[0], a[1], long

# Initialize arrays
mmds = np.full((len(k_bumps), len(k_bumps), len(k_rolls), len(k_rolls), len(long_gs)), None)
accs = np.full((len(k_bumps), len(k_bumps), len(k_rolls), len(k_rolls), len(long_gs)), None)
bals = np.full((len(k_bumps), len(k_bumps), len(k_rolls), len(k_rolls), len(long_gs)), None)
stabs_s = np.full((len(k_bumps), len(k_bumps), len(k_rolls), len(k_rolls), len(long_gs)), None)
stabs_d = np.full((len(k_bumps), len(k_bumps), len(k_rolls), len(k_rolls), len(long_gs)), None)
ctrls_s = np.full((len(k_bumps), len(k_bumps), len(k_rolls), len(k_rolls), len(long_gs)), None)
ctrls_d = np.full((len(k_bumps), len(k_bumps), len(k_rolls), len(k_rolls), len(long_gs)), None)
longs = np.full((len(k_bumps), len(k_bumps), len(k_rolls), len(k_rolls), len(long_gs)), None)
scores = np.full((len(k_bumps), len(k_bumps), len(k_rolls), len(k_rolls), len(long_gs)), None)
final_scores = np.full((len(k_bumps), len(k_bumps), len(k_rolls), len(k_rolls)), None) # avg score across long Gs

# Prepare arguments for the worker function
args = []
for i, kfb in enumerate(k_bumps):
    for j, krb in enumerate(k_bumps):
        for k, kfr in enumerate(k_rolls):
            for l, krr in enumerate(k_rolls):
                for n, long in enumerate(long_gs):
                    args.append((i, j, k, l, n, kfb, krb, kfr, krr, v_avg, long))

# Run using joblib (parallel processing)
results = Parallel(n_jobs=-2, backend="loky")(delayed(process_combination)(*arg) for arg in tqdm(args))


                
print("Finished!")


In [ ]:
# Scoring

# Store the mmd results in the arrays (control calculations will happen later)
for result in results:
    i, j, k, l, g, mmd, ss, sd, cs, cd, a0, a1, ax = result
    mmds[i][j][k][l][g] = mmd
    accs[i][j][k][l][g] = a0
    bals[i][j][k][l][g] = a1
    stabs_s[i][j][k][l][g] = -1 * ss
    stabs_d[i][j][k][l][g] = sd
    ctrls_s[i][j][k][l][g] = cs
    ctrls_d[i][j][k][l][g] = cd
    longs[i][j][k][l][g] = ax

base_a  =    accs[len(k_bumps)//2][len(k_bumps)//2][len(k_rolls)//2][len(k_rolls)//2]
base_ss = stabs_s[len(k_bumps)//2][len(k_bumps)//2][len(k_rolls)//2][len(k_rolls)//2]
base_sd = stabs_d[len(k_bumps)//2][len(k_bumps)//2][len(k_rolls)//2][len(k_rolls)//2]
base_cs = ctrls_s[len(k_bumps)//2][len(k_bumps)//2][len(k_rolls)//2][len(k_rolls)//2]
base_cd = ctrls_d[len(k_bumps)//2][len(k_bumps)//2][len(k_rolls)//2][len(k_rolls)//2]
base_b  =    bals[len(k_bumps)//2][len(k_bumps)//2][len(k_rolls)//2][len(k_rolls)//2]

def eval_score(acc, s_s, s_d, c_s, c_d, b, long_g):
    
    # try to vaguely score the builds
    # A: 10, SS: 0, SD: 5.5, CS: 0, CD: 7, B: 8
    wa = 66.666 # RAW 0.015 --> 1.0
    wss= 0 # RAW 0.17 --> 0
    wsd= 1.666 # RAW 0.33 --> 0.55
    
    # wcs= 0.1/0.07 # RAW 0.07 --> 0.1
    # wcd= 0.2/0.3 # RAW 0.3 --> 0.2
    wcs= 0 # RAW 0.07 --> 0
    wcd= 2.333 # RAW 0.3 --> 0.7
    wb = -0.8/0.55 # RAW 0.55 --> -0.8
    ind = 1
    
    if long_g < 0: # BRAKING
        # A: 4, SS: 8, SD: 3, CS: 7, CD: 4, B: 2
        wa = 0.4/0.015 # about 0.4
        wss = 0.8/0.17 # about 0.8
        wsd = 0.3/.33 # about 0.3
        wcs = 0.7/0.07 # about 0.7
        
        # wcs = 0.85/0.07 # about 0.85
        # wcd = 0.1/0.3 # about 0.1
        wcs = 0.7/0.07 # about 0.7
        wcd = 0.4/0.3 # about 0.4
        # wb = -0.4/0.55 # about -0.4
        wb = -0.2/0.55 # about 0.2
        ind = 0
    elif long_g > 0: # EXIT
        #A: 6, SS: 3, SD: 7, CS: 3, CD: 5.5, B: -4
        wa = 0.6/0.015 # about 0.6
        wss = 0.3/0.17 # about 0.3
        wsd = 0.7/0.33 # about 0.7
        
        # wcs = 0.6/0.07 # about 0.6
        # wcd = 0.1/0.3 # about 0.1
        wcs = 0.3/0.07 # about 0.3
        wcd = 0.55/0.3 # about 0.55
        wb = 0.4/0.55 # about 0.4
        ind = 1
    
    score = 0
    score += (( acc / base_a[ind]) - 1) * wa
    score += (( s_s / base_ss[ind]) - 1) * wss
    score += (( s_d / base_sd[ind]) - 1) * wsd
    score += (( c_s / base_cs[ind]) - 1) * wcs
    score += (( c_d / base_cd[ind]) - 1) * wcd
    score += (( b / base_b[ind]) - 1) * wb
    
    return score

print("TEMP")
print(f"Max: {np.max(accs) / base_a}\tMin: {np.min(accs) / base_a}")
print(f"Max: {np.max(stabs_s) / base_ss}\tMin: {np.min(stabs_s) / base_ss}")
print(f"Max: {np.max(stabs_d) / base_sd}\tMin: {np.min(stabs_d) / base_sd}")
print(f"Max: {np.max(ctrls_s) / base_cs}\tMin: {np.min(ctrls_s) / base_cs}")
print(f"Max: {np.max(ctrls_d) / base_cd}\tMin: {np.min(ctrls_d) / base_cd}")
print(f"Max: {np.max(bals) / base_b}\tMin: {np.min(bals) / base_b}")

# Step 1: Initialize a list to store maximum scores for each longitudinal acceleration
max_scores = np.zeros(len(long_gs)) 
# interesting note: initially this was np.full(len(long_gs), 0), but the array way not updating. 
# This is because it was locked to integers only, but this method tries to assign floats. Python didn't bother throwing an error. 
# Yet another reason why Python's type handling is too lenient.
best_score_i = (0, 0, 0, 0)
best_score = 0

# Step 2: Calculate the maximum scores for each longitudinal acceleration
for i, kfb in enumerate(k_bumps):
    for j, krb in enumerate(k_bumps):
        for k, kfr in enumerate(k_rolls):
            for l, krr in enumerate(k_rolls):
                for g, long in enumerate(long_gs):
                    # Update the maximum score for the given longitudinal acceleration
                    score = eval_score(accs[i][j][k][l][g], stabs_s[i][j][k][l][g], stabs_d[i][j][k][l][g], ctrls_s[i][j][k][l][g], ctrls_d[i][j][k][l][g], bals[i][j][k][l][g], longs[i][j][k][l][g])
                    if score > max_scores[g]:
                        max_scores[g] = score

# Step 3: Normalize scores using the calculated maximums
for i, kfb in enumerate(k_bumps):
    for j, krb in enumerate(k_bumps):
        for k, kfr in enumerate(k_rolls):
            for l, krr in enumerate(k_rolls):
                temp = []
                for g, long in enumerate(long_gs):
                    # Calculate the normalized score
                    raw_score = eval_score(accs[i][j][k][l][g], stabs_s[i][j][k][l][g], stabs_d[i][j][k][l][g], ctrls_s[i][j][k][l][g], ctrls_d[i][j][k][l][g], bals[i][j][k][l][g], longs[i][j][k][l][g])
                    if max_scores[g] != 0:
                        normalized_score = raw_score / max_scores[g]
                    else:
                        normalized_score = 0
                    scores[i][j][k][l][g] = normalized_score
                    temp.append(normalized_score)
                
                # Calculate final score as the average of normalized scores
                final_scores[i][j][k][l] = np.average(temp)
                
                if final_scores[i][j][k][l] > best_score:
                    best_score = final_scores[i][j][k][l]
                    best_score_i = (i, j, k, l)

print(k_bumps)
print(k_rolls)
print("BEST SCORE:")
print(best_score)
print(f"KFB: {k_bumps[best_score_i[0]]}, KRB: {k_bumps[best_score_i[1]]}, KFR: {k_rolls[best_score_i[2]]}, KRR: {k_rolls[best_score_i[3]]}")

In [ ]:
# Alt Implementation (Might be faster)
from joblib import Parallel, delayed
import numpy as np
from tqdm import tqdm

# Define the worker function
def process_combination(i, j, kf, kr, v, ul, mb, md, s, long):
    # Create the car and perform the mmd sweep
    car, _, mmd = gen_car(name=f"KF: {kf}, KR: {kr}", kf=kf, kr=kr)
    mmd.mmd_sweep(v, lin_space=ul, max_beta=mb, max_delta=md, size=s, mu=0.65, long_g=long)
    mmd.clear_high_sa(sa_lut(mmd.v_avg))
    
    # Calculate stability and control values
    stab_s, stab_d, _, _ = mmd.calculate_important_control_values(mode="Stability")
    ctrl_s, ctrl_d, _, _ = mmd.calculate_important_control_values(mode="Control")
    a = mmd.calculate_max_acc()[1]
    
    # Return all relevant results
    return i, j, mmd, stab_s, stab_d, ctrl_s, ctrl_d, a[0], a[1]

# Initialize arrays
mmds = np.full((len(samples), len(samples)), None)
accs = np.full((len(samples), len(samples)), None)
bals = np.full((len(samples), len(samples)), None)
stabs_s = np.full((len(samples), len(samples)), None)
stabs_d = np.full((len(samples), len(samples)), None)
ctrls_s = np.full((len(samples), len(samples)), None)
ctrls_d = np.full((len(samples), len(samples)), None)
scores = np.full((len(samples), len(samples)), None)

# Prepare arguments for the worker function
args = []
for i, fb in enumerate(samples):
    for j, rb in enumerate(samples):
        args.append((i, j, fb, rb, v_avg, use_lin, max_beta, max_delta, size, long_g))

# Run using joblib (parallel processing)
results = Parallel(n_jobs=-2, backend="loky")(delayed(process_combination)(*arg) for arg in tqdm(args))

# Store the results in the arrays
for result in results:
    i, j, mmd, stab_s, stab_d, ctrl_s, ctrl_d, acc, bal = result
    mmds[i][j] = mmd
    stabs_s[i][j] = stab_s
    stabs_d[i][j] = stab_d
    ctrls_s[i][j] = ctrl_s
    ctrls_d[i][j] = ctrl_d
    accs[i][j] = acc
    bals[i][j] = bal

# Try to vaguely score the builds
wa = 100.0 # about 50
wss = 10.0 # about 10
wsd = 5.0 # about 25
wcs = 10.0 # about 10
wcd = 3.0 # about 15
wb = 0.25 # about 5

base_a  = accs[len(samples)//2][len(samples)//2]
base_ss = stabs_s[len(samples)//2][len(samples)//2]
base_sd = stabs_d[len(samples)//2][len(samples)//2]
base_cs = ctrls_s[len(samples)//2][len(samples)//2]
base_cd = ctrls_d[len(samples)//2][len(samples)//2]
base_b  = bals[len(samples)//2][len(samples)//2]

def eval_score(acc, s_s, s_d, c_s, c_d, b):
    score = 0
    score += ( acc / base_a) * wa
    score += ( s_s / base_ss) * wss
    score += ( s_d / base_sd) * wsd
    score += ( c_s / base_cs) * wcs
    score += ( c_d / base_cd) * wcd
    score += ( b / base_b) * wb
    return score

for i in range(len(samples)):
    for j in range(len(samples)):
        scores[i][j] = eval_score(accs[i][j], stabs_s[i][j], stabs_d[i][j], ctrls_s[i][j], ctrls_d[i][j], bals[i][j])

print("Finished!")


In [ ]:
res = [accs, bals, stabs_s, stabs_d, ctrls_s, ctrls_d, final_scores]
titles = ["Peak Ay", "Balance", "Straightline Stability", "Cornering Stability", "Straightline Control", "Cornering Control", "Overall Score (Experimental)"]
axis_titles = ["K F Bump", "K R Bump", "K F Roll", "K R Roll", "Longitudinal Acceleration"]
# axis_titles = ["K F Bump", "K R Bump", "K R Roll", "K F Roll", "Longitudinal Acceleration"] # somehow the order gets swapped? I'm really not sure how

def plot_5d_parameter_sweep(res, titles, samples, const_dims=(0, 0, 0), const_indices=(0, 0, 0)):
    """
    Visualize a 5D parameter sweep by selecting which three dimensions to hold constant.
    Note: the third dimension should be longitudinal acceleration and should not be used as an axis.
    
    Parameters:
    - res: List of 5D (or 4D for 'Overall Score (Experimental)') arrays of results, one array for each metric.
    - titles: Titles of each result for visualization.
    - samples: Values for each dimension.
    - const_dims: A tuple (dim1, dim2, dim3) indicating which dimensions to hold constant (0, 1, 2, 3, or 4).
    - const_indices: A tuple (idx1, idx2, idx3) indicating which indices to select for the constant dimensions.
    """
    # Sort `const_dims` and `const_indices` together in descending order based on dimensions
    ordered_dims = sorted(zip(const_dims, const_indices), reverse=True)
    
    # Dimensions to vary (for heatmap axes)
    var_dims = [i for i in range(5) if i not in const_dims]
    
    for i, axis in enumerate(res):
        # Determine if we are handling "Overall Score (Experimental)"
        is_overall_score = titles[i] == "Overall Score (Experimental)"
        expected_dims = 4 if is_overall_score else 5
        
        data_slice = axis
        print(f"\nProcessing: '{titles[i]}'")
        # print(f"Initial data shape for '{titles[i]}': {data_slice.shape}")

        # Adjust the reduction process if handling 4D data
        for dim, idx in ordered_dims:
            if dim >= expected_dims:
                # Skip reduction for dimensions that don't exist in the data
                print(f"Skipping dimension {dim} for '{titles[i]}' since it is out of bounds.")
                continue

            try:
                data_slice = np.take(data_slice, indices=idx, axis=dim)
                # print(f"After reducing dimension {dim} with index {idx}, data shape: {data_slice.shape}")
            except IndexError as e:
                print(f"IndexError encountered for '{titles[i]}': {e}")
                print(f"Attempted to access axis {dim} but data shape is: {data_slice.shape}")
                continue
        
        # At this point, `data_slice` should be 2D for heatmap plotting
        if data_slice.ndim != 2:
            raise ValueError(f"Expected a 2D slice, but got {data_slice.ndim} dimensions for '{titles[i]}' instead.")
        
        # Normalize the data if applicable
        if not is_overall_score:  # Skip normalization for "Overall Score (Experimental)"
            midpoint_value = data_slice[len(data_slice)//2, len(data_slice[0])//2]
            if midpoint_value != 0:  # Avoid division by zero
                data_slice = np.divide(data_slice, midpoint_value)
        
        # Create the heatmap
        heatmap = go.Figure(
            data=go.Heatmap(
                z=data_slice,
                x=samples[var_dims[0]],
                y=samples[var_dims[1]],
                colorscale='Viridis'
            )
        )
        
        heatmap.update_layout(
            title=titles[i],
            xaxis_title=f'{axis_titles[var_dims[0]]}', 
            yaxis_title=f'{axis_titles[var_dims[1]]}'
        )
        
        heatmap.show()

# Example usage:
# Assume 'inputs' is a list of values for each dimension.
# const_dims specifies the dimensions to hold constant, and const_indices the indices to use for those dimensions.

# Dim order:
# kfb, krb, kfr, krr, long_gs
inputs = [k_bumps, k_bumps, k_rolls, k_rolls, long_gs]
plot_5d_parameter_sweep(res, titles, inputs, const_dims=(0, 1, 4), const_indices=(0, 0, 2))
# plot_5d_parameter_sweep(res, titles, inputs, const_dims=(2, 3, 4), const_indices=(1, 2, 1))

In [ ]:
res = [accs, bals, stabs_s, stabs_d, ctrls_s, ctrls_d, final_scores]
titles = ["Peak Ay", "Balance", "Straightline Stability", "Cornering Stability", "Straightline Control", "Cornering Control", "Overall Score (Experimental)"]

print(res[0])

def plot_5d_parameter_sweep(res, titles, samples, const_dims=(0, 0, 0), const_indices=(0, 0, 0)):
    """
    Visualize a 5D parameter sweep by selecting which three dimensions to hold constant.
    Note: the third dimension should be longitudinal acceleration and should not be used as an axis.
    
    Parameters:
    - res: List of 5D arrays of results, one array for each metric.
    - titles: Titles of each result for visualization.
    - samples: Values for each dimension.
    - const_dims: A tuple (dim1, dim2, dim3) indicating which dimensions to hold constant (0, 1, 2, 3, or 4).
    - const_indices: A tuple (idx1, idx2, idx3) indicating which indices to select for the constant dimensions.
    """
    # Sort `const_dims` and `const_indices` together in descending order based on dimensions
    ordered_dims = sorted(zip(const_dims, const_indices), reverse=True)
    
    # Dimensions to vary (for heatmap axes)
    var_dims = [i for i in range(5) if i not in const_dims]
    
    for i, axis in enumerate(res):
        # Extract a 2D slice from the 5D data based on the constant dimensions
        data_slice = axis
        
        # Apply reductions in descending order of dimensions
        for dim, idx in ordered_dims:
            data_slice = np.take(data_slice, indices=idx, axis=dim)
        
        # At this point, `data_slice` should be 2D for heatmap plotting
        if data_slice.ndim != 2:
            raise ValueError(f"Expected a 2D slice, but got {data_slice.ndim} dimensions instead.")
        
        # Normalize the data if applicable
        if titles[i] != "Overall Score (Experimental)":
            midpoint_value = data_slice[len(data_slice)//2, len(data_slice[0])//2]
            if midpoint_value != 0:  # Avoid division by zero
                data_slice = np.divide(data_slice, midpoint_value)
        
        # Create the heatmap
        heatmap = go.Figure(
            data=go.Heatmap(
                z=data_slice,
                x=samples[var_dims[0]],
                y=samples[var_dims[1]],
                colorscale='Viridis'
            )
        )
        
        heatmap.update_layout(
            title=titles[i],
            xaxis_title=f'% Change Dimension {var_dims[0] + 1}', 
            yaxis_title=f'% Change Dimension {var_dims[1] + 1}'
        )
        
        heatmap.show()

# Example usage:
# Assume 'inputs' is a list of values for each dimension.
# const_dims specifies the dimensions to hold constant, and const_indices the indices to use for those dimensions.

# Dim order:
# kfb, krb, kfr, krr, long_gs
inputs = [k_bumps, k_bumps, k_rolls, k_rolls, long_gs]
plot_5d_parameter_sweep(res, titles, inputs, const_dims=(0, 1, 4), const_indices=(0, 0, 1))

In [ ]:
X, Y = np.meshgrid(samples, samples)

# accs, bals, etc are all now 4D arrays instead of 2D arrays, and cannot be swept in 2D. So you must pick two axes to display.

res = [accs, bals, stabs_s, stabs_d, ctrls_s, ctrls_d, final_scores]
titles = ["Peak Ay", "Balance", "Straightline Stability", "Cornering Stability", "Straightline Control", "Cornering Control", "Overall Score (Experimental)"]

for i, axis in enumerate(res):
    # print(f"Printing {titles[i]}")
    # print(axis)
    if titles[i] != "Overall Score (Experimental)":
        heatmap = go.Figure(
            data = go.Heatmap(
                z=np.divide(axis, axis[len(axis)//2][len(axis)//2]),
                x=samples,
                y=samples,
                colorscale='Viridis'
            )
        )
    else:
        heatmap = go.Figure(
            data = go.Heatmap(
                z=axis,
                x=samples,
                y=samples,
                colorscale='Viridis'
            )
        )
    
    heatmap.update_layout(
        title=titles[i],
        xaxis_title=f'% Front Stiffness',
        yaxis_title=f'% Rear Stiffness'
    )
    
    heatmap.show()

In [ ]:
# long sweep

mmds = []

samples = np.linspace(-1.7*9.81, 1.2*9.81, 12)
accs = []

for val in samples:
    print(val)
    car, _, mmd = gen_car(name = f"2D MMD at {val:.2f}")
    
    mmd.mmd_sweep(v_avg, lin_space=use_lin, max_beta=max_beta, max_delta=max_delta, size=size, mu=0.65, long_g=val)
    a = mmd.calculate_max_acc()
    mmd.clear_high_sa(sa_lut(mmd.v_avg))
    accs.append(a)
    mmds.append(mmd)

In [ ]:
print(f"GG Sweep at {v_avg} m/s")
for i, a in enumerate(accs):
    print(f"Ay: {np.round(a[1][0], 3)} Gs \t Ax: {np.round(samples[i]/9.81, 3)} Gs")

In [ ]:
# compare
mmds2 = []
# same samples
for val in samples:
    car, _, mmd = gen_car(k_r = 1.3)
    # car, _, mmd = gen_car(front_track = 47.6*IN_TO_M, rear_track = 47*IN_TO_M)
    mmd.mmd_sweep(val, lin_space=use_lin, max_beta=max_beta, max_delta=max_delta, size=size, mu=0.65, long_g=0)
    mmds2.append(mmd)

In [ ]:
mmd_list = []

# print(max(np.array(mmds[0].ay).flatten()))
for i, m in enumerate(mmds):
    # m.clear_high_sa(sa_lut(m.v_avg))
    m.plot_mmd(pub=True, lat=3.0, use_name=True)
    # mmd_list.append(m)

In [ ]:
mmd_list2 = []
# print(max(np.array(mmds2[0].ay).flatten()))
for m in mmds2:
    m.clear_high_sa(sa_lut(m.v_avg))
    m.plot_mmd(pub=True, lat=2.5)
    mmd_list2.append(m)

In [ ]:
#mmds.sort(key=lambda x:x.v_avg)
plot_accel_vs_vel(mmd_list, mmd_list2 = mmd_list2)
plot_accel_vs_vel(mmd_list2)